In [ ]:
import  os
import  tensorflow as tf
import  numpy as np
import utils 
from tensorflow import keras
from utils import LoadFishDataUtil
from tensorflow.keras.preprocessing.image import ImageDataGenerator,array_to_img,img_to_array,load_img


In [ ]:
data_dir ='/media/xingbo/Storage/fish_identification/data/SESSION_AQUARIUM/SESSION_TRAIN'
BATCH_SIZE = 8
SPLIT_WEIGHTS = (0.7, 0.15, 0.15) # Data prepare split wight train vs validation vs test
IMG_WIDTH=320
IMG_HEIGHT=60

In [ ]:
myloadData = LoadFishDataUtil(data_dir,BATCH_SIZE,IMG_WIDTH,IMG_HEIGHT)
train_dataset,val_dataset,test_dataset,STEPS_PER_EPOCH, CLASS_NAMES,class_num = myloadData.loadFishData()


In [ ]:

# 3x3 convolution
def conv3x3(channels, stride=1, kernel=(3, 3)):
    return keras.layers.Conv2D(channels, kernel, strides=stride, padding='same',
                               use_bias=False,
                            kernel_initializer=tf.random_normal_initializer())

class ResnetBlock(keras.Model):

    def __init__(self, channels, strides=1, residual_path=False):
        super(ResnetBlock, self).__init__()

        self.channels = channels
        self.strides = strides
        self.residual_path = residual_path

        self.conv1 = conv3x3(channels, strides)
        self.bn1 = keras.layers.BatchNormalization()
        self.conv2 = conv3x3(channels)
        self.bn2 = keras.layers.BatchNormalization()

        if residual_path:
            self.down_conv = conv3x3(channels, strides, kernel=(1, 1))
            self.down_bn = tf.keras.layers.BatchNormalization()

    def call(self, inputs, training=None):
        residual = inputs

        x = self.bn1(inputs, training=training)
        x = tf.nn.relu(x)
        x = self.conv1(x)
        x = self.bn2(x, training=training)
        x = tf.nn.relu(x)
        x = self.conv2(x)

        # this module can be added into self.
        # however, module in for can not be added.
        if self.residual_path:
            residual = self.down_bn(inputs, training=training)
            residual = tf.nn.relu(residual)
            residual = self.down_conv(residual)

        x = x + residual
        return x


class ResNet(keras.Model):

    def __init__(self, block_list, num_classes, initial_filters=16, **kwargs):
        super(ResNet, self).__init__(**kwargs)

        self.num_blocks = len(block_list)
        self.block_list = block_list

        self.in_channels = initial_filters
        self.out_channels = initial_filters
        self.conv_initial = conv3x3(self.out_channels)

        self.blocks = keras.models.Sequential(name='dynamic-blocks')

        # build all the blocks
        for block_id in range(len(block_list)):
            for layer_id in range(block_list[block_id]):

                if block_id != 0 and layer_id == 0:
                    block = ResnetBlock(self.out_channels, strides=2, residual_path=True)
                else:
                    if self.in_channels != self.out_channels:
                        residual_path = True
                    else:
                        residual_path = False
                    block = ResnetBlock(self.out_channels, residual_path=residual_path)

                self.in_channels = self.out_channels

                self.blocks.add(block)

            self.out_channels *= 2

        self.final_bn = keras.layers.BatchNormalization()
        self.avg_pool = keras.layers.GlobalAveragePooling2D()
        self.fc = keras.layers.Dense(num_classes)

    def call(self, inputs, training=None):

        out = self.conv_initial(inputs)

        out = self.blocks(out, training=training)

        out = self.final_bn(out, training=training)
        out = tf.nn.relu(out)

        out = self.avg_pool(out)
        out = self.fc(out)


        return out

# In[3]:

In [ ]:
num_classes = class_num
epochs = 20
validation_steps = 20
# build model and optimizer
model = ResNet([2, 2, 2], num_classes)
model.compile(optimizer=keras.optimizers.Adam(0.001),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),#TripletSemiHardLoss
              metrics=['accuracy'])
model.build(input_shape=(None, IMG_WIDTH,IMG_HEIGHT, 3))


In [ ]:
print("Number of variables in the model :", len(model.variables))
model.summary()
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
# This callback will stop the training when there is no improvement in
# the validation loss for three consecutive epochs.
# train
history=model.fit(train_dataset, epochs=epochs,callbacks=[callback],
          validation_data=val_dataset, verbose=1, validation_steps=validation_steps)

# evaluate on test set
scores = model.evaluate(test_dataset, verbose=1)
print("Final test loss and accuracy :", scores)

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

## Save model

In [ ]:
# Save the model
# model.save('vggfish.h5py')

In [ ]:
model.save_weights('resnetfish20191120.h5')

In [ ]:
model.load_weights('resnetfish20191120.h5')

In [ ]:

# evaluate on test set
scores = model.evaluate(test_dataset, verbose=1)
print("Final test loss and accuracy :", scores)

In [ ]:
test_data_dir ='/media/xingbo/Storage/fish_identification/data/SESSION_AQUARIUM/SESSION2'
BATCH_SIZE = 8
testloadData = LoadFishDataUtil(test_data_dir,BATCH_SIZE,IMG_WIDTH,IMG_HEIGHT,CLASS_NAMES)
sess2_test_dataset,sess2_class_num = testloadData.loadTestFishData()

sess2_test_dataset

In [ ]:
loss0,accuracy0 = model.evaluate(sess2_test_dataset, steps = 20)

In [ ]:
from tensorflow.keras.preprocessing import image
#img = image.load_img('/media/xingbo/Storage/fish_identification/data/SESSION_AQUARIUM/SESSION1/02883A/fish_2_02883A__4.png', target_size=(IMG_WIDTH,IMG_HEIGHT))
img = image.load_img('/media/xingbo/Storage/fish_identification/data/SESSION_AQUARIUM/SESSION2/02883A/fish_2_02883A__03.png', target_size=(IMG_WIDTH,IMG_HEIGHT))
#img = image.load_img('/media/xingbo/Storage/fish_identification/data/SESSION_AQUARIUM/SESSION3/02883A/fish_2_02883A_04.png', target_size=(IMG_WIDTH,IMG_HEIGHT))
#plt.figure()
#plt.imshow(img)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x=x/255
predictions = model(x)
indxmax=np.argmax(predictions)
print('predictions:', CLASS_NAMES[indxmax] )


for x,y in sess2_train_dataset:
    predictions = model.predict(x)
    #print(predictions[0])
    #print('predictions shape:', predictions.shape)
    print('x shape:', x.shape)
    for n in range(8):
        indxmax=np.argmax(predictions[n])
        #print('predictions max index:',indxmax)
        print('predictions:', CLASS_NAMES[indxmax] )
        print('real:', CLASS_NAMES[np.argwhere(y[n]).ravel()] )
       